# [Module 4.2] 캠페인 호출을 통한  추천 리스트 얻기

이 노트북은 캠페인을 통해서 실제 추천 결과를 받아 봅니다.
두개의 캠페인을 통하여 추천 리스트를 받습니다. 
- 첫번째는 user-personalization 으로 user_id를 제공하면 유저에 적당한 추천 리스트를 받습니다.
- 두번째는 sims 로 item_id를 제공하면 유사한 아이템을 추천 합니다.



In [1]:
import boto3
import json
import numpy as np
import pandas as pd
import time
from datetime import datetime

다음으로 여러분의 환경이 Amazon Personalize와 성공적으로 통신할 수 있는지 확인해야 합니다.

In [2]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

아래 코드 셀은 이전 notebook에서 저장했던 공유 변수들을 불러옵니다.

In [10]:
%store -r

생성할 오브젝트의 끝에 임의의 숫자를 부여하기 위해 suffix 정의

## 추천 결과 얻기

캠페인이 활성화되면 추천 결과를 받을 수 있습니다. 먼저 컬렉션에서 임의의 사용자를 선택해야 합니다. 그런 다음, ID 대신 추천을 위해 영화 정보를 표시하는 몇 가지 헬퍼 함수를 만듭니다.

#### Help 함수

In [5]:
items_all = pd.read_csv('./ml-1m/movies.dat',sep='::', encoding='latin1',names=['ITEM_ID', 'TITLE', 'GENRE'],)
items=items_all.copy()

items['to_keep'] = items['ITEM_ID'].apply(lambda x:x in unique_items)
items=items[items['to_keep']]
del items['to_keep']
items.tail()

#len(unique_items)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


,ITEM_ID,TITLE,GENRE
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama
3882,3952,"Contender, The (2000)",Drama|Thriller


In [7]:
def get_movie_title(movie_id):
    """
    Takes in an ID, returns a title
    """
    movie_id = int(movie_id)
    movie_title=items[items['ITEM_ID']==movie_id]['TITLE']
    return (movie_title.tolist())

def get_movie_genre(movie_id):
    """
    Takes in an ID, returns a genre
    """
    movie_id = int(movie_id)
    movie_genre=items[items['ITEM_ID']==movie_id]['GENRE']
    return (movie_genre.tolist())


## User-Personalization GetRecommendations 호출

아래 코드 셀을 실행하면 특정 사용자에 대한 추천 사항이 표시되고 추천 영화 목록이 반환됩니다.

In [14]:
user_personalization_campaign_arn='arn:aws:personalize:us-east-1:603420654815:campaign/Movielens-user-personalization-campaign-31639'
sims_campaign_arn='arn:aws:personalize:us-east-1:603420654815:campaign/Movielens-sims-campaign-31639'

In [8]:
# Help 함수
def rec_item_list(item_list):
    recommendation_title_list = []
    recommendation_genre_list = []
    # recommendation_id_list=[]
    for item in item_list:
        title = get_movie_title(item['itemId'])
        genre = get_movie_genre(item['itemId'])
        recommendation_title_list.append(title)
        recommendation_genre_list.append(genre)    
        
    recommendations_df = pd.DataFrame(data={'OriginalRecs':recommendation_title_list,
                                            'Genre': recommendation_genre_list})
    return recommendations_df                                                

In [12]:
df=pd.read_csv(warm_train_interaction_filename)

# Getting a random user:
user_id, item_id, _,_,_ = df.sample().values[0]
user_id = 4375 # 특정 유저 선택, 결과 확인 후에 추후에 주석 표시하여 랜더 유저 선택할 수 있습니다.
print("USER: {}".format(user_id))

get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = user_personalization_campaign_arn,    
    userId = str(user_id),
)
# Update DF rendering
pd.set_option('display.max_rows', 30)

print("Recommendations for user: ", user_id)

item_list = get_recommendations_response['itemList']

recommendations_df = rec_item_list(item_list)
recommendations_df

USER: 4375
Recommendations for user:  4375


,OriginalRecs,Genre
0,"[Nightmare on Elm Street 3: Dream Warriors, A ...",[Horror]
1,[Friday the 13th (1980)],[Horror]
2,[Ghostbusters II (1989)],[Comedy|Horror]
3,[Child's Play (1988)],[Horror]
4,[Pet Sematary (1989)],[Horror]
5,[Poltergeist II: The Other Side (1986)],[Horror|Thriller]
6,[Gremlins (1984)],[Comedy|Horror]
7,"[Fog, The (1980)]",[Horror]
8,[Candyman (1992)],[Horror]
9,"[Believers, The (1987)]",[Horror|Thriller]


## Sims GetRecommendations 호출
아래 코드 셀을 실행하면 특정 아이템과 유사한 추천 영화 목록이 반환됩니다.

In [15]:
# Getting a random item_id:
_, item_id, _,_,_ = df.sample().values[0]
title = get_movie_title(item_id)
genre = get_movie_genre(item_id)

print("Recommendations for item_id: {}, Title:{}, Genre:{}".format(item_id,title[0], genre[0]))

# print("Movie title: {}, Genre:{} ".format(title[0], genre[0]))


get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = sims_campaign_arn,
    itemId = str(item_id),
)
# Update DF rendering
pd.set_option('display.max_rows', 30)



item_list = get_recommendations_response['itemList']
recommendations_df = rec_item_list(item_list)
recommendations_df


Recommendations for item_id: 3481, Title:High Fidelity (2000), Genre:Comedy


,OriginalRecs,Genre
0,[American Psycho (2000)],[Comedy|Horror|Thriller]
1,[Magnolia (1999)],[Drama]
2,[Keeping the Faith (2000)],[Comedy|Romance]
3,"[Big Kahuna, The (2000)]",[Comedy|Drama]
4,[Rules of Engagement (2000)],[Drama|Thriller]
5,[Frequency (2000)],[Drama|Thriller]
6,[Me Myself I (2000)],[Comedy]
7,[Mission to Mars (2000)],[Sci-Fi]
8,[Meet the Parents (2000)],[Comedy]
9,[Chicken Run (2000)],[Animation|Children's|Comedy]


다음 노트북의 실시간 추천에 사용하기 위해서 아래를 저장 합니다.

In [16]:
%store recommendations_df
%store user_id

Stored 'recommendations_df' (DataFrame)
Stored 'user_id' (int)
